In [16]:
!pip install mxnet

In [5]:
import pandas as pd
import matplotlib.pyplot as plt

from mxnet import nd, gluon, init, autograd
from mxnet.gluon import nn
import mxnet as mx
from mxnet.gluon.data.vision import datasets, transforms
from IPython import display
import matplotlib.pyplot as plt
import time

from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score
from katonic.ml import MyClient

In [6]:
# Load data from a CSV file into a PandasDataset
myclient = MyClient()
mlflow = myclient.mlflow
client = myclient.client

mnist_train = datasets.FashionMNIST(train=True)
X, y = mnist_train[0]
('X shape: ', X.shape, 'X dtype', X.dtype, 'y:', y)

('X shape: ', (28, 28, 1), 'X dtype', numpy.uint8, 'y:', 2)

In [7]:
def acc(output, label):
    # output: (batch, num_output) float32 ndarray
    # label: (batch, ) int32 ndarray
    return (output.argmax(axis=1) ==
            label.astype('float32')).mean().asscalar()

In [9]:
exp_name = "mlflow-test-gluon"
mlflow.set_experiment(exp_name)
exp_details = mlflow.get_experiment_by_name(exp_name)
args = {
    'prediction_length': 12,
    'freq': 'M',
    'max_epochs':5
}
with mlflow.start_run(run_name=exp_name):
    transformer = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(0.13, 0.31)
    ])
    mnist_train = mnist_train.transform_first(transformer)
    batch_size = 256
    train_data = gluon.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=4)
    
    mnist_valid = gluon.data.vision.FashionMNIST(train=False)
    valid_data = gluon.data.DataLoader(
        mnist_valid.transform_first(transformer),
        batch_size=batch_size, num_workers=4)
    
    net = nn.HybridSequential()
    net.add(nn.Conv2D(channels=6, kernel_size=5, activation='relu'),
            nn.MaxPool2D(pool_size=2, strides=2),
            nn.Conv2D(channels=16, kernel_size=3, activation='relu'),
            nn.MaxPool2D(pool_size=2, strides=2),
            nn.Flatten(),
            nn.Dense(120, activation="relu"),
            nn.Dense(84, activation="relu"),
            nn.Dense(10))
    net.hybridize()
    net.initialize()#init=init.Xavier(), ctx=mx.cpu(0))
    
    softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
    trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1})
    
    for epoch in range(10):
        train_loss, train_acc, valid_acc = 0., 0., 0.
        tic = time.time()
        for data, label in train_data:
            # forward + backward
            with autograd.record():
                output = net(data)
                loss = softmax_cross_entropy(output, label)
            loss.backward()
            # update parameters
            trainer.step(batch_size)
            # calculate training metrics
            train_loss += loss.mean().asscalar()
            train_acc += acc(output, label)
        # calculate validation accuracy
        for data, label in valid_data:
            valid_acc += acc(net(data), label)
        print("Epoch %d: loss %.3f, train acc %.3f, test acc %.3f, in %.1f sec" % (
                epoch, train_loss/len(train_data), train_acc/len(train_data),
                valid_acc/len(valid_data), time.time()-tic))

    model_info = mlflow.gluon.log_model(gluon_model=net, artifact_path="model")
    

MXNetError: MXNetError: Error in operator normalize2_normalize0: [11:48:18] ../src/operator/image/./image_random-inl.h:251: Check failed: nchannels == 3 || nchannels == 1: The first dimension of input tensor must be the channel dimension with either 1 or 3 elements, but got input with shape [28,1,28]